In [2]:
! pip install torch torchvision onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 2.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pillow-11.1.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 1.7 MB/s eta 0:

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Гиперпараметры
batch_size = 64
learning_rate = 0.01
epochs = 5

# Загрузка данных
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Определение модели
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Разворачиваем изображение
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Создание модели и оптимизатора
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(epochs):
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Сохранение обученной модели
torch.save(model.state_dict(), "mnist_model.pth")

100.0%
100.0%
100.0%
100.0%


Epoch [1/5], Loss: 0.4725
Epoch [2/5], Loss: 0.2371
Epoch [3/5], Loss: 0.0780
Epoch [4/5], Loss: 0.2122
Epoch [5/5], Loss: 0.3836


In [5]:
dummy_input = torch.randn(1, 1, 28, 28) 
torch.onnx.export(model, dummy_input, "mnist_model.onnx", input_names=["input"], output_names=["output"])
print("Модель успешно экспортирована в ONNX.")

Модель успешно экспортирована в ONNX.


In [6]:
import onnxruntime as ort
import numpy as np
import torch

# Загрузим ONNX-модель
session = ort.InferenceSession("mnist_model.onnx")

# Тестовый ввод
dummy_input = torch.randn(1, 1, 28, 28).numpy()

# Запуск предсказания
outputs = session.run(None, {"input": dummy_input})
print("Выход модели:", outputs)

Выход модели: [array([[-221.3485  ,  -53.577065,  -13.014393,   87.604065, -140.17885 ,
         -14.636501, -195.58873 ,  -11.833436,  -76.045074,  -57.548992]],
      dtype=float32)]
